In [39]:
import numpy as np
import librosa
import librosa.display
import soundfile as sf

# Example usage
audio_file_path = '../inputAudio/marine.mp3' 
audio_file_path = '../outputAudio/marine/vocals.wav' 
output_file_path = '3-3low=to-high.wav'  # Replace with your desired output file path

In [36]:
def analyze_audio(audio_file_path, rms_window_length=0.02, rms_hop_length=0.01, 
                  change_threshold_factor=2.5, cooldown_time=0.2, low_level_percentile=25):
    # Load the audio file
    y, sr = librosa.load(audio_file_path, sr=None)

    # Calculate the RMS energy for each frame
    frame_length = int(rms_window_length * sr)
    hop_length = int(rms_hop_length * sr)
    rms_energy = librosa.feature.rms(y=y, frame_length=frame_length, hop_length=hop_length)[0]

    # Smooth the RMS energy using a moving average
    smooth_rms_energy = np.convolve(rms_energy, np.ones(3)/3, mode='valid')

    # Calculate the change in RMS energy between consecutive frames
    rms_change = np.diff(smooth_rms_energy)

    # Define 'low' RMS energy level as a percentile of the RMS energy
    low_rms_level = np.percentile(smooth_rms_energy, low_level_percentile)

    # Define a threshold for significant RMS change
    rms_change_threshold = np.median(np.abs(rms_change)) * change_threshold_factor

    # Find points where RMS change is above the threshold and the previous RMS value is below the 'low' level
    significant_changes = np.where((rms_change > rms_change_threshold) & 
                                   (smooth_rms_energy[:-1] < low_rms_level))[0]

    # Convert frame numbers to time and apply cooldown (debouncing)
    significant_change_times = []
    last_time = 0
    for frame in significant_changes:
        time = librosa.frames_to_time(frame + 1, sr=sr, hop_length=hop_length)
        if time - last_time > cooldown_time:  # Apply cooldown period
            significant_change_times.append(time)
            last_time = time

    return significant_change_times



# Replace 'path_to_audio_file.mp3' with the path to your MP3 file
onset_times = analyze_audio(audio_file_path)
print(onset_times)


[0.56, 3.03, 4.72, 9.15, 13.87, 23.22, 23.54, 24.65, 25.48, 25.9, 27.29, 32.13, 33.52, 33.79, 35.02, 40.98, 42.08, 42.38, 42.95, 44.52, 45.14, 49.84, 50.4, 51.24, 53.4, 55.68, 55.94, 56.75, 61.2, 63.67, 63.97, 64.54, 65.65, 66.06, 68.12, 70.92, 75.06, 75.62, 76.75, 78.95, 79.46, 82.27, 85.88, 86.14, 86.98, 87.25, 88.64, 92.19, 93.63, 94.74, 95.84, 97.23, 106.35, 107.75, 108.51, 116.89, 118.11, 124.04, 126.05, 127.69, 128.53, 129.23, 129.91, 130.46, 131.01, 132.96, 133.5, 134.88, 138.79, 144.3, 146.8, 148.75, 149.16, 151.23, 151.51, 157.54, 158.15, 158.69, 159.74, 161.96, 162.58, 165.37, 166.9, 168.96, 169.52, 170.09, 175.34, 175.87, 176.73, 177.56, 178.11, 178.94, 180.03, 193.49, 194.35, 195.28, 198.01, 199.91, 201.96, 205.26, 206.36, 206.78, 208.85, 209.14, 210.67, 211.63, 214.88, 215.16, 215.78, 216.06, 216.32, 216.62, 219.54, 220.18, 222.15, 222.4, 222.99, 226.6, 226.87, 228.81, 232.92, 235.19, 235.45, 235.73, 236.28, 236.56, 237.38, 239.87, 240.72, 244.59, 245.69, 247.1, 251.24, 25

In [38]:


def generate_beep(duration=0.1, frequency=1000.0, sample_rate=22050):
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    beep = np.sin(2 * np.pi * frequency * t) * 0.5
    return beep

def create_beep_audio(onset_times, sample_rate=22050, output_file_path='beep_audio.wav'):
    if not onset_times:
        raise ValueError("Onset times array is empty")

    # Generate a beep sound
    beep_duration = 0.1  # duration of the beep in seconds
    beep = generate_beep(duration=beep_duration, frequency=1000.0, sample_rate=sample_rate)

    # Calculate the duration of the audio based on the last onset time
    max_onset_time = max(onset_times)
    audio_duration = max_onset_time + beep_duration

    # Create a silent audio signal of the required length
    silent_audio = np.zeros(int(audio_duration * sample_rate))

    # Insert the beep at each onset time
    for onset_time in onset_times:
        start_sample = int(onset_time * sample_rate)
        end_sample = start_sample + len(beep)
        
        # Ensure the beep doesn't go past the end of the audio
        if end_sample <= len(silent_audio):
            silent_audio[start_sample:end_sample] += beep
        else:
            # If the beep would go past the end, just add what can fit
            silent_audio[start_sample:] += beep[:len(silent_audio) - start_sample]
            
    # Write the beep audio signal to the output file
    sf.write(output_file_path, silent_audio, sample_rate)

create_beep_audio(onset_times, output_file_path=output_file_path)


In [40]:
from Utility import music_ops
music_ops.get_beeptrack_from_timestamps('/outputAudio/marine/vocals.wav', '3-3low=to-high.wav')


ModuleNotFoundError: No module named 'Utility'